In [1]:
# Needed for SQL Alchemy  
from sqlalchemy import create_engine
engine=create_engine('sqlite:///.CourseData',echo=False)

# All query results are returned as Pandas DataFrames 
import pandas as pd

#Used for importing CSVs
import glob
import os

# Needed for %sql Magic
%load_ext sql

import sqlite3


In [2]:
#Create CourseData and CourseDqtaWarehouse Databases
conn = sqlite3.connect("./CourseData.db")
conn = sqlite3.connect("./CourseDataWarehouse.db")

In [3]:
CourseCatalog2017_2018=pd.read_csv("./SourceData/Catalogs/CourseCatalog2017_2018.csv")
CourseCatalog2018_2019=pd.read_csv("./SourceData/Catalogs/CourseCatalog2018_2019.csv")

In [4]:
#Reading and combining all course meetings CSVs

#Get list of folders/files in Source Data folder
folders=glob.glob('./SourceData/*')

#Delete folders/files from list that don't contain course meeting CSV files
del folders[5]
del folders[12]
del folders[16]

#Read CSVs from folders in list and combine into a single dataframe
all_files=[]
for path in folders:
    all_files += glob.glob(os.path.join(path, "course_meetings.csv"))     # advisable to use os.path.join as this makes concatenation OS independent
df_from_each_file = (pd.read_csv(f) for f in all_files)
course_meetings = pd.concat(df_from_each_file, ignore_index=True, sort=True)

course_meetings.term.unique()


array(['Fall2015', 'Fall2014', 'Winter2015', 'Spring2015', 'Summer2015',
       'Fall2016', 'Fall2018', 'Fall2017', 'Spring2019',
       'SpringBreak2017', 'Spring2017', 'Winter2018', 'Winter2016',
       'Spring2016', 'Spring2018', 'Winter2017', 'Summer2017',
       'Summer2018', 'Summer2016'], dtype=object)

In [5]:
#Reading and combining all courses CSVs

#Get list of folders/files in Source Data folder
folders=glob.glob('./SourceData/*')

#Delete folders/files from list that don't contain courses CSV files
del folders[5]
del folders[12]
del folders[16]

#Read CSVs from folders in list and combine into a single dataframe
all_files=[]
for path in folders:
    all_files += glob.glob(os.path.join(path, "courses.csv"))     # advisable to use os.path.join as this makes concatenation OS independent
df_from_each_file = (pd.read_csv(f) for f in all_files)
courses = pd.concat(df_from_each_file, ignore_index=True, sort=True)

courses.term.unique()



array(['Fall2015', 'Fall2014', 'Winter2015', 'Spring2015', 'Summer2015',
       'Fall2016', 'Fall2018', 'Fall2017', 'Spring2019',
       'SpringBreak2017', 'Spring2017', 'Winter2018', 'Winter2016',
       'Spring2016', 'Spring2018', 'Winter2017', 'Summer2017',
       'Summer2018', 'Summer2016'], dtype=object)

In [6]:
CourseCatalog2017_2018.to_sql('CourseCatalog2017_2018',con=engine,if_exists='replace')
CourseCatalog2018_2019.to_sql('CourseCatalog2018_2019',con=engine,if_exists='replace')

In [7]:
#Check length of course_meetings DataFrame
len(course_meetings.index)

317321

In [8]:
#import course_meetings DataFrame to SQL as separate tables because DataFrame is too large to import at once.
#Will use a union later in SQL to combine them

course_meetings[:35000].to_sql('course_meetings1',con=engine,if_exists='replace')
course_meetings[35000:70000].to_sql('course_meetings2',con=engine,if_exists='replace')
course_meetings[70000:105000].to_sql('course_meetings3',con=engine,if_exists='replace')
course_meetings[105000:140000].to_sql('course_meetings4',con=engine,if_exists='replace')
course_meetings[140000:175000].to_sql('course_meetings5',con=engine,if_exists='replace')
course_meetings[175000:210000].to_sql('course_meetings6',con=engine,if_exists='replace')
course_meetings[210000:245000].to_sql('course_meetings7',con=engine,if_exists='replace')
course_meetings[245000:280000].to_sql('course_meetings8',con=engine,if_exists='replace')
course_meetings[280000:315000].to_sql('course_meetings9',con=engine,if_exists='replace')
course_meetings[315000:].to_sql('course_meetings10',con=engine,if_exists='replace')

In [9]:
#Check length of courses DataFrame
len(courses.index)

15937

In [10]:
courses.to_sql('courses',con=engine,if_exists='replace')

In [11]:
# Initialize 
%sql sqlite:///.CourseData

'Connected: @.CourseData'

In [12]:
%%sql

SELECT name, sql FROM sqlite_master
WHERE type='table'
ORDER BY name;

 * sqlite:///.CourseData
Done.


name,sql
CourseCatalog2017_2018,"CREATE TABLE ""CourseCatalog2017_2018"" ( ""index"" BIGINT, program_code TEXT, program_name TEXT, catalog_id TEXT, course_title TEXT, credits TEXT, prereqs TEXT, coreqs TEXT, fees TEXT, attributes TEXT, description TEXT)"
CourseCatalog2018_2019,"CREATE TABLE ""CourseCatalog2018_2019"" ( ""index"" BIGINT, program_code TEXT, program_name TEXT, catalog_id TEXT, course_title TEXT, credits TEXT, prereqs TEXT, coreqs TEXT, fees TEXT, attributes TEXT, description TEXT)"
all_course_meetings,"CREATE TABLE all_course_meetings (term text,crn int,location datexttatype,day text,start text,end text)"
course_meetings1,"CREATE TABLE course_meetings1 ( ""index"" BIGINT, crn BIGINT, day TEXT, ""end"" TEXT, location TEXT, start TEXT, term TEXT)"
course_meetings10,"CREATE TABLE course_meetings10 ( ""index"" BIGINT, crn BIGINT, day TEXT, ""end"" TEXT, location TEXT, start TEXT, term TEXT)"
course_meetings2,"CREATE TABLE course_meetings2 ( ""index"" BIGINT, crn BIGINT, day TEXT, ""end"" TEXT, location TEXT, start TEXT, term TEXT)"
course_meetings3,"CREATE TABLE course_meetings3 ( ""index"" BIGINT, crn BIGINT, day TEXT, ""end"" TEXT, location TEXT, start TEXT, term TEXT)"
course_meetings4,"CREATE TABLE course_meetings4 ( ""index"" BIGINT, crn BIGINT, day TEXT, ""end"" TEXT, location TEXT, start TEXT, term TEXT)"
course_meetings5,"CREATE TABLE course_meetings5 ( ""index"" BIGINT, crn BIGINT, day TEXT, ""end"" TEXT, location TEXT, start TEXT, term TEXT)"
course_meetings6,"CREATE TABLE course_meetings6 ( ""index"" BIGINT, crn BIGINT, day TEXT, ""end"" TEXT, location TEXT, start TEXT, term TEXT)"


In [13]:
%%sql

CREATE TABLE all_course_meetings (
term text,
crn int,
location datexttatype,
day text,
start text,
end text
);

 * sqlite:///.CourseData
(sqlite3.OperationalError) table all_course_meetings already exists [SQL: 'CREATE TABLE all_course_meetings (\nterm text,\ncrn int,\nlocation datexttatype,\nday text,\nstart text,\nend text\n);'] (Background on this error at: http://sqlalche.me/e/e3q8)


In [14]:
%%sql

INSERT INTO all_course_meetings
    select term,crn,location,day,start,end from course_meetings1
union all
    select term,crn,location,day,start,end from course_meetings2
union all
    select term,crn,location,day,start,end from course_meetings3
union all
    select term,crn,location,day,start,end from course_meetings4
union all
    select term,crn,location,day,start,end from course_meetings5
union all
    select term,crn,location,day,start,end from course_meetings6
union all
    select term,crn,location,day,start,end from course_meetings7
union all
    select term,crn,location,day,start,end from course_meetings8
union all
    select term,crn,location,day,start,end from course_meetings9
union all
    select term,crn,location,day,start,end from course_meetings10
;

 * sqlite:///.CourseData
317321 rows affected.


[]

In [15]:
%%sql

select * from all_course_meetings limit 10


 * sqlite:///.CourseData
Done.


term,crn,location,day,start,end
Fall2015,75222,DSB 112,R,2015-09-03T14:00:00,2015-09-03T15:15:00
Fall2015,75222,DSB 112,M,2015-09-07T14:00:00,2015-09-07T15:15:00
Fall2015,75222,DSB 112,R,2015-09-10T14:00:00,2015-09-10T15:15:00
Fall2015,75222,DSB 112,M,2015-09-14T14:00:00,2015-09-14T15:15:00
Fall2015,75222,DSB 112,R,2015-09-17T14:00:00,2015-09-17T15:15:00
Fall2015,75222,DSB 112,M,2015-09-21T14:00:00,2015-09-21T15:15:00
Fall2015,75222,DSB 112,R,2015-09-24T14:00:00,2015-09-24T15:15:00
Fall2015,75222,DSB 112,M,2015-09-28T14:00:00,2015-09-28T15:15:00
Fall2015,75222,DSB 112,R,2015-10-01T14:00:00,2015-10-01T15:15:00
Fall2015,75222,DSB 112,M,2015-10-05T14:00:00,2015-10-05T15:15:00


In [28]:
%%sql

alter table all_course_meetings add MeetingDate date;
update all_course_meetings set MeetingDate = substr(start,1,10)


 * sqlite:///.CourseData
Done.
634642 rows affected.


[]

In [31]:
##Break Start and End columns in all_course_meetings tables into separate date and time columns
%%sql

alter table all_course_meetings add StartTime time;
update all_course_meetings set StartTime=substr(start,12);
alter table all_course_meetings add EndTime time;
update all_course_meetings set EndTime=substr(end,12);

select * from all_course_meetings limit 10;

 * sqlite:///.CourseData
(sqlite3.OperationalError) duplicate column name: StartTime [SQL: 'alter table all_course_meetings add StartTime time;'] (Background on this error at: http://sqlalche.me/e/e3q8)


In [37]:
%%sql

select * from courses limit 10;



 * sqlite:///.CourseData
Done.


index,act,cap,catalog_id,credits,crn,meetings,primary_instructor,rem,section,term,timecodes,title
0,35,0,AC 0011,3.000,75222,"[{'days': 'MR', 'times': '0200pm-0315pm', 'dates': '09/01-12/07', 'location': 'DSB 112'}]",Rebecca I. Bloch,-35,C01,Fall2015,['MR 0200pm-0315pm 09/01-12/07 DSB 112'],Introduction to Financial Accounting
1,33,0,AC 0011,3.000,75240,"[{'days': 'MR', 'times': '0330pm-0445pm', 'dates': '09/01-12/07', 'location': 'DSB 112'}]",Rebecca I. Bloch,-33,C02,Fall2015,['MR 0330pm-0445pm 09/01-12/07 DSB 112'],Introduction to Financial Accounting
2,32,0,AC 0011,3.000,75241,"[{'days': 'TF', 'times': '0200pm-0315pm', 'dates': '09/01-12/07', 'location': 'DSB 115'}]",Michael P. Coyne,-32,C03,Fall2015,['TF 0200pm-0315pm 09/01-12/07 DSB 115'],Introduction to Financial Accounting
3,35,0,AC 0011,3.000,75243,"[{'days': 'TF', 'times': '0800am-0915am', 'dates': '09/01-12/07', 'location': 'DSB 111'}]",Michael P. Coyne,-35,C04,Fall2015,['TF 0800am-0915am 09/01-12/07 DSB 111'],Introduction to Financial Accounting
4,36,0,AC 0011,3.000,75244,"[{'days': 'TF', 'times': '0930am-1045am', 'dates': '09/01-12/07', 'location': 'DSB 111'}]",Michael P. Coyne,-36,C05,Fall2015,['TF 0930am-1045am 09/01-12/07 DSB 111'],Introduction to Financial Accounting
5,32,29,AC 0011,3.000,75242,"[{'days': 'MR', 'times': '0930am-1045am', 'dates': '09/01-12/07', 'location': 'DSB 111'}]",Paul Caster,-3,F,Fall2015,['MR 0930am-1045am 09/01-12/07 DSB 111'],Introduction to Financial Accounting
6,30,29,AC 0011,3.000,75461,"[{'days': 'TF', 'times': '1230pm-0145pm', 'dates': '09/01-12/07', 'location': 'DSB 111'}]",Jo Ann Drusbosky,-1,G,Fall2015,['TF 1230pm-0145pm 09/01-12/07 DSB 111'],Introduction to Financial Accounting
7,32,29,AC 0011,3.000,75484,"[{'days': 'TF', 'times': '0330pm-0445pm', 'dates': '09/01-12/07', 'location': 'DSB 111'}]",Jo Ann Drusbosky,-3,I,Fall2015,['TF 0330pm-0445pm 09/01-12/07 DSB 111'],Introduction to Financial Accounting
8,28,29,AC 0011,3.000,75729,"[{'days': 'TF', 'times': '1100am-1215pm', 'dates': '09/01-12/07', 'location': 'DSB 105'}]",Arleen N. Kardos,1,J,Fall2015,['TF 1100am-1215pm 09/01-12/07 DSB 105'],Introduction to Financial Accounting
9,29,29,AC 0011,3.000,76101,"[{'days': 'TF', 'times': '1230pm-0145pm', 'dates': '09/01-12/07', 'location': 'DSB 105'}]",Arleen N. Kardos,0,K,Fall2015,['TF 1230pm-0145pm 09/01-12/07 DSB 105'],Introduction to Financial Accounting
